In [62]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from flask import request, jsonify
from pytesseract import pytesseract
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
from sklearn.model_selection import train_test_split

In [63]:
# Define the path to the T5 model
model_path = r'D:\Personal\Edu\OCR-Expresso\t5_gec_model' 

In [64]:
# Load the pre-trained T5 model and tokenizer
loaded_model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

In [67]:
def grammar_corrector(input_text, num_return_sequences):
    # Tokenize the input text
    batch = tokenizer([input_text], truncation=True, padding='max_length', max_length=64, return_tensors="pt")
    
    # Generate corrected text using the loaded model
    translated = loaded_model.generate(**batch, max_length=64, num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
    
    # Decode the generated text
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    
    # Join the list of strings into a single string
    result_string = ' '.join(tgt_text)
    
    return result_string 

In [68]:
df = pd.read_csv('D:/Personal/Edu/Study/FYP/NLP/data_set/c4_200m_550k.csv')
df.shape

(550000, 2)

In [75]:
train_df, test_df = train_test_split(df, test_size=0.1, shuffle=False)
train_df.shape, test_df.shape

((549450, 2), (550, 2))

In [76]:
def evaluate_model(model, tokenizer, dataset):
    # Initialize lists to store predictions and ground truth labels
    all_predictions = []
    all_labels = []

    # Iterate through the dataset
    for example in dataset:
        # Extract input text
        input_text = example['input']
        
        # Generate prediction using the model
        prediction = grammar_corrector(input_text, num_return_sequences=1)
        #print(prediction)
        # Extract ground truth label
        ground_truth = example['output']
        
        # Append prediction and ground truth to the respective lists
        all_predictions.append(prediction)
        all_labels.append(ground_truth)
    
    # Compute confusion matrix
    cm = confusion_matrix(all_labels, all_predictions)
    
    # Compute accuracy
    accuracy = accuracy_score(all_labels, all_predictions)
    
    # Compute precision
    precision = precision_score(all_labels, all_predictions, average='weighted')
    
    # Compute recall
    recall = recall_score(all_labels, all_predictions, average='weighted')
    
    # Compute F1 score
    f1 = f1_score(all_labels, all_predictions, average='weighted')
    
    return cm, accuracy, precision, recall, f1

In [77]:
test_data = test_df.to_dict(orient='records')

In [ ]:
conf_matrix, accuracy, precision, recall, f1 = evaluate_model(loaded_model, tokenizer, test_data)

C:\Users\PradeepWijayawickram\anaconda3\Lib\site-packages\transformers\generation\configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [56]:
print("Confusion Matrix:")
print(conf_matrix)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Confusion Matrix:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]]
Accuracy: 0.11454545454545455
Precision: 0.11454545454545455
Recall: 0.11454545454545455
F1 Score: 0.11454545454545455
